In [1]:
# Check PyTorch version installed on this system
!python -c "import torch; print(torch.__version__)"

1.13.0+cu116


In [2]:
!nvidia-smi

Thu Dec 15 23:43:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   30C    P0    52W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Download the required modules
"""
Assign to TORCH with what you get from the cell above, E.g., export TORCH=1.13.0+cu116
"""
%env TORCH=1.13.0+cu116
!pip install torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install torch-geometric
!pip install torchmetrics
!pip install matplotlib
!pip install networkx
!pip install scipy

env: TORCH=1.13.0+cu116
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-1.13.0+cu116.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
github_username="deeplearningtester"
github_repository="grl"
github_token = ""

%env github_repository={github_repository}

env: github_repository=grl


In [5]:
%cd $github_repository

/content/grl


In [6]:
import torch
import random

random.seed(42)
torch.manual_seed(42)

In [7]:
from torch_geometric.datasets import PPI
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import AddSelfLoops
import dataset

train_dataset = PPI(root='', split='train', transform=AddSelfLoops())
val_dataset = PPI(root='', split='val', transform=AddSelfLoops())
test_dataset = PPI(root='', split='test', transform=AddSelfLoops())

dataset.num_features = 50
dataset.num_labels = 121
dataset.train_loader = DataLoader(train_dataset, batch_size=2)
dataset.val_loader = DataLoader(val_dataset, batch_size=2)
dataset.test_loader = DataLoader(test_dataset, batch_size=2)

In [8]:
from evaluation import evaluate
from training_loop import train

In [9]:
from models import GATv1PPI
model = GATv1PPI(dataset.num_features, dataset.num_labels)
evaluate(model, dataset.test_loader)

(tensor(0.7045, device='cuda:0'), tensor(0.3902, device='cuda:0'))

In [10]:
ppi_train_params = {
  "lr": 5e-3,
  "weight_decay": 0,
  "epochs": 400,
  "patience": 100,
  "model_name": model.model_name
}

In [11]:
def run_experiment(iters=10):
  losses = []
  scores = []
  for iter in range(iters):
    best_model, _, _, _, _, _ = train(model, ppi_train_params, verbose=False)
    loss, score = evaluate(best_model, dataset.test_loader)
    losses.append(loss)
    scores.append(score)
  losses = torch.tensor(losses)
  scores = torch.tensor(scores)
  return (torch.std_mean(losses), torch.std_mean(scores))

In [12]:
loss_ci, score_ci = run_experiment(10)

training model GATv1PPI
GATv1PPI(
  (layers): ModuleList(
    (0): GATLayer()
    (1): GATLayer()
    (2): GATLayer(
      (act): Identity()
    )
  )
)
training...
best model performance @ epoch 00268: 
	val_loss: 0.1089 | val_micro_f1: 0.9638
	test_loss: 0.0621 | test_micro_f1: 0.9790
training model GATv1PPI
GATv1PPI(
  (layers): ModuleList(
    (0): GATLayer()
    (1): GATLayer()
    (2): GATLayer(
      (act): Identity()
    )
  )
)
training...
best model performance @ epoch 00179: 
	val_loss: 0.1064 | val_micro_f1: 0.9598
	test_loss: 0.0650 | test_micro_f1: 0.9738
training model GATv1PPI
GATv1PPI(
  (layers): ModuleList(
    (0): GATLayer()
    (1): GATLayer()
    (2): GATLayer(
      (act): Identity()
    )
  )
)
training...
best model performance @ epoch 00214: 
	val_loss: 0.1192 | val_micro_f1: 0.9574
	test_loss: 0.0659 | test_micro_f1: 0.9737
training model GATv1PPI
GATv1PPI(
  (layers): ModuleList(
    (0): GATLayer()
    (1): GATLayer()
    (2): GATLayer(
      (act): Identi

In [13]:
loss_std, loss_mean = loss_ci
score_std, score_mean = score_ci

print(f'loss:\t\t{loss_mean:.4f} +/- {loss_std:.4f}')
print(f'micro F1 score: {score_mean:.4f} +/- {score_std:.4f}')

loss:		0.0617 +/- 0.0039
micro F1 score: 0.9753 +/- 0.0022
